![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [106]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [107]:
# Replace the '?'s with NaN in dataset
cc_apps_nans_replaced = cc_apps.replace("?", np.NaN)

In [108]:
# Create a copy of the NaN replacement DataFrame
cc_apps_imputed = cc_apps_nans_replaced.copy()

In [109]:
# Iterate over each column of cc_apps_nans_replaced and impute the most frequent value for object data types and the mean for numeric data types
for col in cc_apps_imputed.columns:
    # Check if the column is of object type
    if cc_apps_imputed[col].dtypes == "object":
        # Impute with the most frequent value
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(
            cc_apps_imputed[col].value_counts().index[0]
        )
    else:
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(cc_apps_imputed[col].mean())


In [110]:
cc_apps_dummies = pd.get_dummies(cc_apps_imputed, drop_first=True)

In [111]:
cc_apps_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Columns: 383 entries, 2 to 13_-
dtypes: float64(2), int64(2), uint8(379)
memory usage: 277.1 KB


In [112]:
cc_apps_dummies.columns

Index([        2,         7,        10,        12,     '0_b', '1_15.17',
       '1_15.75', '1_15.83', '1_15.92', '1_16.00',
       ...
           '6_j',     '6_n',     '6_o',     '6_v',     '6_z',     '8_t',
           '9_t',    '11_p',    '11_s',    '13_-'],
      dtype='object', length=383)

In [113]:
X = cc_apps_dummies.iloc[:, :-1].values
y = cc_apps_dummies.iloc[:, [-1]].values

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)

In [115]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [116]:
logreg = LogisticRegression()
logreg.fit(scaled_X_train, y_train)
y_pred = logreg.predict(scaled_X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[66 26]
 [22 93]]


In [117]:
# Define the grid of values for tol and max_iter
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

In [118]:
# Create a dictionary where tol and max_iter are keys and the lists of their values are the corresponding values
param_grid = {
    'tol': [1e-4, 1e-3, 1e-2],
    'max_iter': [100, 200, 500]
}

In [119]:
# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

In [120]:
# Fit the GridSearchCV
grid_model = grid_model.fit(scaled_X_train, y_train)

In [121]:
# Summarize results
best_train_score, best_train_params = grid_model.best_score_, grid_model.best_params_
print("Best: %f using %s" % (best_train_score, best_train_params))

Best: 0.815765 using {'max_iter': 100, 'tol': 0.01}


In [124]:
# Extract the best model and evaluate it on the test set
best_model = grid_model.best_estimator_
best_score =  best_model.score(scaled_X_test, y_test)

print("Accuracy of logistic regression classifier: ", best_score)

LogisticRegression(tol=0.01)
Accuracy of logistic regression classifier:  0.7681159420289855
